In [1]:
import pandas as pd
import pymysql
curr_dir = '/home/souche/qiongjiu/hgc/'

In [2]:
# 读取数据 
car_data_columns= ["country_id","manufacturer","battery_quality_mile","battery_quality_time","brand_code","brand_name","car_body",
"cylinder_number","driving_mode","engine_volume_l","fuel_form","gear_box_type","guide_price","height","import_type",
"intake_type","length","level","max_power","max_torque","model_code","model_name","quality_mile","quality_time","rate",
"rate_count","seat_number_top","series_code","series_level","series_name","wheel_base","width","year","down_market_time",
"production_time","shutdown_time","time_to_market","year_1","year_2","year_3","year_4","year_5","year_6","year_7","year_8",
"year_9","year_10","year_11","year_12","year_13","year_14","year_15","year_16"]
q_res = pd.read_csv(curr_dir+'all_car_model_sep_word_to_sql',header=None,sep='\t',names=car_data_columns)
q_res = q_res[['model_code','series_code','brand_code']]
q_res.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 65251 entries, 0 to 65250
Data columns (total 3 columns):
model_code     65251 non-null object
series_code    65251 non-null object
brand_code     65251 non-null object
dtypes: object(3)
memory usage: 1.5+ MB


/home/souche/projects/calculation/py3dev_new/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (2,3,7,9,12,13,16,18,19,20,22,23,24,25,26,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
# 读取车辆级别
data_level = pd.read_csv(curr_dir+"iEnterprise2.8_car_parameter_onehot.csv", header = 0, low_memory=False)
data_level = data_level[['model_code','level']]
data_level.info()
data_level=pd.merge(data_level,q_res,how='left',on='model_code')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 65250 entries, 0 to 65249
Data columns (total 2 columns):
model_code    65250 non-null object
level         65250 non-null object
dtypes: object(2)
memory usage: 1019.6+ KB


In [4]:
# A级车、B级车分开
data_level_A = data_level[(data_level['level']=="缺失") | 
                          (data_level['level']=="A00") | 
                          (data_level['level']=="A0") | 
                          (data_level['level']=="A")]

data_level_B = data_level[(data_level['level']=="B") | 
                          (data_level['level']=="C") | 
                          (data_level['level']=="D") | 
                          (data_level['level']=="E")]
print(data_level_A.shape)
print(data_level_B.shape)

(40573, 4)
(24677, 4)


In [5]:
#读取计算完成的零售数据A级车和B级车的精度
retail_A = pd.read_csv(curr_dir+"2020-11-02采购数据-A级车-误差中位数.csv", header = 0, low_memory=False)
retail_A.info()

retail_B = pd.read_csv(curr_dir+"2020-11-02采购数据-B级车-误差中位数.csv", header = 0, low_memory=False)
retail_B.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13454 entries, 0 to 13453
Data columns (total 2 columns):
model_code    13454 non-null object
median_res    13454 non-null float64
dtypes: float64(1), object(1)
memory usage: 210.3+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8895 entries, 0 to 8894
Data columns (total 2 columns):
model_code    8895 non-null object
median_res    8895 non-null float64
dtypes: float64(1), object(1)
memory usage: 139.1+ KB


In [6]:
data_A = pd.merge(data_level_A, retail_A, how='left', on='model_code')
data_B = pd.merge(data_level_B, retail_B, how='left', on='model_code')

data_A = data_A[['brand_code', 'series_code', 'model_code', 'median_res', 'level']]
data_B = data_B[['brand_code', 'series_code', 'model_code', 'median_res', 'level']]

data_A.info()
data_B.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 40573 entries, 0 to 40572
Data columns (total 5 columns):
brand_code     31070 non-null object
series_code    31070 non-null object
model_code     40573 non-null object
median_res     13454 non-null float64
level          40573 non-null object
dtypes: float64(1), object(4)
memory usage: 1.9+ MB
<class 'pandas.core.frame.DataFrame'>
Int64Index: 24677 entries, 0 to 24676
Data columns (total 5 columns):
brand_code     17796 non-null object
series_code    17796 non-null object
model_code     24677 non-null object
median_res     8895 non-null float64
level          24677 non-null object
dtypes: float64(1), object(4)
memory usage: 1.1+ MB


In [7]:
# 按车系填充
series_A = pd.DataFrame(columns={'series_code':str, 'median':float})
series_A['series_code'] = list(data_A.groupby(['series_code'])['median_res'].mean().index)
series_A['median'] = list(data_A.groupby(['series_code'])['median_res'].mean())

series_B = pd.DataFrame(columns={'series_code':str, 'median':float})
series_B['series_code'] = list(data_B.groupby(['series_code'])['median_res'].mean().index)
series_B['median'] = list(data_B.groupby(['series_code'])['median_res'].mean())

In [8]:
data_A = pd.merge(data_A, series_A, how='left', on='series_code')
data_A.info()

data_B = pd.merge(data_B, series_B, how='left', on='series_code')
data_B.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 40573 entries, 0 to 40572
Data columns (total 6 columns):
brand_code     31070 non-null object
series_code    31070 non-null object
model_code     40573 non-null object
median_res     13454 non-null float64
level          40573 non-null object
median         26263 non-null float64
dtypes: float64(2), object(4)
memory usage: 2.2+ MB
<class 'pandas.core.frame.DataFrame'>
Int64Index: 24677 entries, 0 to 24676
Data columns (total 6 columns):
brand_code     17796 non-null object
series_code    17796 non-null object
model_code     24677 non-null object
median_res     8895 non-null float64
level          24677 non-null object
median         17003 non-null float64
dtypes: float64(2), object(4)
memory usage: 1.3+ MB


In [9]:
def fill_median(tar):
    if(pd.isnull(tar['median_res'])):
        if(pd.notnull(tar['median'])):
            tar['median_res'] = tar['median']
    return tar

data_AA = data_A.apply(fill_median, axis=1)
data_BB = data_B.apply(fill_median, axis=1)

data_AA.info()
data_BB.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 40573 entries, 0 to 40572
Data columns (total 6 columns):
brand_code     31070 non-null object
series_code    31070 non-null object
model_code     40573 non-null object
median_res     27219 non-null float64
level          40573 non-null object
median         26263 non-null float64
dtypes: float64(2), object(4)
memory usage: 2.2+ MB
<class 'pandas.core.frame.DataFrame'>
Int64Index: 24677 entries, 0 to 24676
Data columns (total 6 columns):
brand_code     17796 non-null object
series_code    17796 non-null object
model_code     24677 non-null object
median_res     17697 non-null float64
level          24677 non-null object
median         17003 non-null float64
dtypes: float64(2), object(4)
memory usage: 1.3+ MB


In [10]:
data_AA['median_res'] = data_AA['median_res'].fillna(retail_A['median_res'].mean())
data_AA.info()

data_BB['median_res'] = data_BB['median_res'].fillna(retail_B['median_res'].mean())
data_BB.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 40573 entries, 0 to 40572
Data columns (total 6 columns):
brand_code     31070 non-null object
series_code    31070 non-null object
model_code     40573 non-null object
median_res     40573 non-null float64
level          40573 non-null object
median         26263 non-null float64
dtypes: float64(2), object(4)
memory usage: 2.2+ MB
<class 'pandas.core.frame.DataFrame'>
Int64Index: 24677 entries, 0 to 24676
Data columns (total 6 columns):
brand_code     17796 non-null object
series_code    17796 non-null object
model_code     24677 non-null object
median_res     24677 non-null float64
level          24677 non-null object
median         17003 non-null float64
dtypes: float64(2), object(4)
memory usage: 1.3+ MB


In [11]:
data_AAA = data_AA[['model_code', 'median_res']]
data_BBB = data_BB[['model_code', 'median_res']]

data_AB = pd.concat([data_AAA, data_BBB])
data_AB.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 65250 entries, 0 to 24676
Data columns (total 2 columns):
model_code    65250 non-null object
median_res    65250 non-null float64
dtypes: float64(1), object(1)
memory usage: 1.5+ MB


In [12]:
# 读取count数据
data = pd.read_csv(curr_dir+"2020-11-02版带count处理完成的采购数据m.csv", header = 0, low_memory=False)
data_counts = data[['model_code', 'count']].drop_duplicates()

data_AB = pd.merge(data_AB, data_counts, how='left', on="model_code")
data_AB.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 65250 entries, 0 to 65249
Data columns (total 3 columns):
model_code    65250 non-null object
median_res    65250 non-null float64
count         25300 non-null float64
dtypes: float64(2), object(1)
memory usage: 2.0+ MB


In [13]:
data_AB = data_AB.rename(columns = {"count":"data_number"})
data_AB.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 65250 entries, 0 to 65249
Data columns (total 3 columns):
model_code     65250 non-null object
median_res     65250 non-null float64
data_number    25300 non-null float64
dtypes: float64(2), object(1)
memory usage: 2.0+ MB


In [16]:
data_AB.to_csv(curr_dir+"2020-11-02采购模型的置信度.csv", header = True, index = False)

In [18]:
ph = curr_dir+"enterprise2_purchase_model_confidence.csv"
data_AB.to_csv(ph)
from dayu.hooks.oss_hook import OSSHook
oss = OSSHook("oss_algorithm")
oss.put_file("algorithm/qiongjiu/valuation/2020-11-02/enterprise2_purchase_model_confidence.csv", ph)


DAYU_HOME : /home/souche/projects/datacenter-etl-v2


/home/souche/projects/calculation/py3dev_new/lib/python3.5/site-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


[2020-11-04 10:51:07,120] {driver:120} INFO - Generating grammar tables from /usr/lib/python3.5/lib2to3/Grammar.txt
[2020-11-04 10:51:07,148] {driver:120} INFO - Generating grammar tables from /usr/lib/python3.5/lib2to3/PatternGrammar.txt
[2020-11-04 10:51:07,480] {oss_hook:28} INFO - Done. Loaded the key algorithm/qiongjiu/valuation/2020-11-02/enterprise2_purchase_model_confidence.csv .
